# Appendix

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [4]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [5]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\ekp0303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [6]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [7]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [8]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [9]:
# import pycap
import redcap
import json
import pandas as pd
# import requests

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [10]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [11]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

## Filter Data During Export

REDCapR and PyCap functions have options for filtering data upon export. If a REDCap project has a large amount of data that is slow to export, then we recommend using REDCapR/PyCap functions to filter the data during export when applicable.

Creating custom reports within REDCap and then exporting those filtered reports is another option for filtering the data before export for projects with a large amount of data. See @sec-reports.

### Filter By Record ID

Export data for Record IDs 1 and 2.

::: {.panel-tabset}

#### R

In [9]:
%%capture
%%R
data_by_record <- redcap_read_oneshot(
    records = c(1,2), 
    redcap_uri = url, 
    token = token
)$data

::: {.content-hidden when-format="html"}

In [10]:
%%R
data_by_record_tbl <- gt(head(data_by_record))
gt::gtsave(data_by_record_tbl, filename = 'export_records_filtered_1.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="300" src="./files/export_files/export_records_filtered_1.html" title="Quarto Documentation"></iframe></p>

#### Python

In [14]:
project.export_records(records=['1','2'],
                       raw_or_label='label',
                       format_type='df').reset_index()

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,first_name,last_name,phone_num,zip_code,dob,age,...,cc_phone,cc_email,close_contacts_complete,supervisor_name,supervisor_email,work_inperson_yesno,work_date,work_contagious,work_contagious_calc,work_information_complete
0,1,Personal Info,NaN,NaN,John,Doe,(999) 999-9999,98105.0,2006-04-11,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Boss,NaN,No,NaN,No,NaN,Complete
2,1,Case Intake,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Notifications,Close Contacts,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Notifications,Close Contacts,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,Personal Info,NaN,NaN,Jane,Doe,(999) 999-9999,98105.0,1994-06-29,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,Notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Boss,fake_email@gmail.com,Yes,2023-10-10,Yes,NaN,Complete
7,2,Case Intake,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,Case Intake,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,Notifications,Close Contacts,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN


:::

### Filter By Date

Data exports can be filtered by the date the record was added or modified. In the following example, we will view all records that were modified or added after April 9, 2024. For more information on how these records were modified, refer to the logging section of this guide. See @sec-logging. 

::: {.panel-tabset}

#### R

In [12]:
%%capture
%%R
data_by_date <- redcap_read_oneshot(
    redcap_uri = url, 
    token = token, 
    datetime_range_begin = as.POSIXct("2024/04/09", 
                                    format = "%Y/%m/%d")
)$data


**Note:** Need to specify date format as seen above using [as.POSIXct](https://rdrr.io/r/base/as.POSIXlt.html)

::: {.content-hidden when-format="html"}

In [13]:
%%R
data_by_date_tbl <- gt(data_by_date)
gt::gtsave(data_by_date_tbl, filename = 'export_records_filtered_2.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="300" src="./files/export_files/export_records_filtered_2.html" title="Quarto Documentation"></iframe></p>

#### Python

In [11]:
from datetime import datetime
project.export_records(date_begin=datetime.fromisoformat("2024-04-09"), 
                       format_type='df')

redcap_repeat_instrument  \
record_id redcap_event_name                              
1         personal_info_arm_1                      NaN   
          notifications_arm_1                      NaN   
          case_intake_arm_1                        NaN   
          notifications_arm_1           close_contacts   
          notifications_arm_1           close_contacts   
2         personal_info_arm_1                      NaN   
          notifications_arm_1                      NaN   
          case_intake_arm_1                        NaN   
          case_intake_arm_1                        NaN   
          notifications_arm_1           close_contacts   
          notifications_arm_1           close_contacts   
3         personal_info_arm_1                      NaN   
          notifications_arm_1                      NaN   
          case_intake_arm_1                        NaN   
          notifications_arm_1           close_contacts   
          notifications_arm_1           close_contacts   

                               redcap_repeat_instance first_name last_name  \
record_id redcap_event_name                                                  
1         personal_info_arm_1                     NaN       John       Doe   
          notifications_arm_1                     NaN        NaN       NaN   
          case_intake_arm_1                       1.0        NaN       NaN   
          notifications_arm_1                     1.0        NaN       NaN   
          notifications_arm_1                     2.0        NaN       NaN   
2         personal_info_arm_1                     NaN       Jane       Doe   
          notifications_arm_1                     NaN        NaN       NaN   
          case_intake_arm_1                       1.0        NaN       NaN   
          case_intake_arm_1                       2.0        NaN       NaN   
          notifications_arm_1                     1.0        NaN       NaN   
          notifications_arm_1                     2.0        NaN       NaN   
3         personal_info_arm_1                     NaN       John       Doe   
          notifications_arm_1                     NaN        NaN       NaN   
          case_intake_arm_1                       1.0        NaN       NaN   
          notifications_arm_1                     1.0        NaN       NaN   
          notifications_arm_1                     2.0        NaN       NaN   

                                    phone_num  zip_code         dob   age  \
record_id redcap_event_name                                                 
1         personal_info_arm_1  (999) 999-9999   98105.0  2006-04-11  18.0   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          case_intake_arm_1               NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
2         personal_info_arm_1  (999) 999-9999   98105.0  1994-06-29  29.0   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          case_intake_arm_1               NaN       NaN         NaN   NaN   
          case_intake_arm_1               NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
3         personal_info_arm_1  (999) 999-9999   98105.0         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          case_intake_arm_1               NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   
          notifications_arm_1             NaN       NaN         NaN   NaN   

                               ethnicity  race  ...        cc_phone  \
record_id redcap_event_name                     ...                   
1         personal_info_arm_1        1.0   4.0  ...

:::

### Filter By Field Value

**Export records that reported 'female' for gender.**

::: {.panel-tabset}

#### R

In [14]:
%%capture
%%R
data_by_gender <- redcap_read_oneshot(
    redcap_uri = url, 
    token = token,
    filter_logic = "[gender] = '0'" 
)$data %>%
 select(c(record_id, gender))

::: {.content-hidden when-format="html"}

In [15]:
%%R
data_by_gender_tbl <- gt(head(data_by_gender))
gt::gtsave(data_by_gender_tbl, filename = 'export_records_filtered_3.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="200" src="./files/export_files/export_records_filtered_3.html" title="Quarto Documentation"></iframe></p>

**Note:** When filtering on REDCap's multiple choice variables, yes/no variables, and checkboxes, you must put quotes around the coded value when using the `filter_logic` argument, otherwise REDCap will not perform the filtering correctly. 

#### Python

In [17]:
project.export_records(filter_logic="[gender] = 0", 
                       format_type='df'
                      ).reset_index()[['record_id', 'gender']]

,record_id,gender
0,2,0
1,4,0
2,6,0


:::

**Export records that reported an age greater than 20.**

::: {.panel-tabset}

#### R

In [21]:
%%capture
%%R
data_by_age <- redcap_read_oneshot(
    redcap_uri = url, 
    token = token, 
    filter_logic = "[age] > 20 " 
)$data%>%
 select(c(record_id, age))

::: {.content-hidden when-format="html"}

In [22]:
%%R
data_by_age_tbl <- gt(head(data_by_age))
gt::gtsave(data_by_age_tbl, filename = 'export_records_filtered_4.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="230" src="./files/export_files/export_records_filtered_4.html" title="Quarto Documentation"></iframe></p>

**Note:** Because age is a numeric field in REDCap, it does not need quotes around the number 20.

#### Python

In [20]:
project.export_records(filter_logic="[age] > 20", 
                       format_type='df'
                      ).reset_index()[['record_id', 'age']]

,record_id,age
0,2,29
1,4,28
2,5,34
3,6,25


:::

## Export Selected Fields

**Export first and last name**

::: {.panel-tabset}

#### R

In [18]:
%%capture
%%R
#specifying record_id automatically also pulls the event, instrument and instance columns (when applicable).
field_subset_1 <- redcap_read_oneshot(
    fields = c("record_id","first_name","last_name"), 
    redcap_uri = url, 
    token = token
)$data

::: {.content-hidden when-format="html"}

In [20]:
%%capture
%%R
field_subset_1_tbl<- gt(head(field_subset_1))
gt::gtsave(field_subset_1_tbl, filename = 'export_records5.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="300" src="./files/export_files/export_records5.html" title="Quarto Documentation"></iframe></p>

**Note:** if `record_id` is not specified, no identifier fields will be exported. By including `record_id` in the `fields` argument, all variables that make up the unique key are automatically exported. 

#### Python

In [21]:
project.export_records(records=['3','4'], 
                       fields=["first_name","last_name"],
                       format_type='df')

redcap_repeat_instrument  \
record_id redcap_event_name                               
3         personal_info_arm_1                       NaN   
          notifications_arm_1                       NaN   
          case_intake_arm_1                         NaN   
4         personal_info_arm_1                       NaN   
          notifications_arm_1                       NaN   
          case_intake_arm_1                         NaN   
          case_intake_arm_1                         NaN   

                               redcap_repeat_instance first_name last_name  
record_id redcap_event_name                                                 
3         personal_info_arm_1                     NaN       John       Doe  
          notifications_arm_1                     NaN        NaN       NaN  
          case_intake_arm_1                       1.0        NaN       NaN  
4         personal_info_arm_1                     NaN       Jane       Doe  
          notifications_arm_1                     NaN        NaN       NaN  
          case_intake_arm_1                       1.0        NaN       NaN  
          case_intake_arm_1                       2.0        NaN       NaN

**Note:** if `record_id` is not specified, all fields that make up the unique key will still be exported.

:::

## Export Specific Instruments

See @sec-forms_instruments to get a list of all the instrument names in your project. Specifying instruments to export will still export all rows of the project data (including rows not relevant to the desired instrument). However, it is useful because it will only export the fields (columns) in that instrument. Use the `filter_logic` argument and the `<form>_complete` variable (automatically created by REDCap for each form) to get the desired output. 

In this example, we use the `symptoms_complete` field to export the 'symptoms' form and all associated data.

::: {.panel-tabset}

#### R

In [29]:
%%capture
%%R

records_dem <- redcap_read_oneshot(redcap_uri = url, 
                           fields = "record_id",
                           forms = "symptoms", 
                           filter_logic = '[symptoms_complete] <> ""',
                           token = token
                                       )$data

records_dem_tbl <- gt(head(records_dem))
gt::gtsave(records_dem_tbl, filename = 'export_records6.html', path = "./files/export_files/")

::: {.content-hidden when-format="html"}

In [16]:
%%capture
%%R

records_dem_tbl <- gt(records_dem)
gt::gtsave(records_dem_tbl, filename = 'export_records6.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="300" src="./files/export_files/export_records6.html" title="Quarto Documentation"></iframe></p>

**Note:** You must add `record_id` to the `fields` argument for the data to export with the `record_id` and associated unique key attached.

#### Python

In [24]:
project.export_records(forms='symptoms',
                       filter_logic ="[symptoms_complete] <>''",
                       format_type='df')

redcap_repeat_instrument  redcap_repeat_instance  \
record_id redcap_event_name                                                     
1         case_intake_arm_1                       NaN                       1   
2         case_intake_arm_1                       NaN                       1   
          case_intake_arm_1                       NaN                       2   
3         case_intake_arm_1                       NaN                       1   
4         case_intake_arm_1                       NaN                       1   
          case_intake_arm_1                       NaN                       2   
5         case_intake_arm_1                       NaN                       1   
6         case_intake_arm_1                       NaN                       1   

                             symptoms_yesno symptom_onset  symptoms_exp___1  \
record_id redcap_event_name                                                   
1         case_intake_arm_1               1    2023-10-09                 1   
2         case_intake_arm_1               1    2023-10-11                 1   
          case_intake_arm_1               1    2021-06-05                 1   
3         case_intake_arm_1               1    2023-10-09                 1   
4         case_intake_arm_1               1    2023-10-11                 1   
          case_intake_arm_1               1    2021-06-05                 1   
5         case_intake_arm_1               1    2023-10-08                 1   
6         case_intake_arm_1               1    2023-10-02                 0   

                             symptoms_exp___2  symptoms_exp___3  \
record_id redcap_event_name                                       
1         case_intake_arm_1                 1                 0   
2         case_intake_arm_1                 1                 0   
          case_intake_arm_1                 1                 1   
3         case_intake_arm_1                 1                 0   
4         case_intake_arm_1                 1                 0   
          case_intake_arm_1                 1                 1   
5         case_intake_arm_1                 0                 0   
6         case_intake_arm_1                 0                 0   

                             symptoms_exp___4  symptoms_exp___5  \
record_id redcap_event_name                                       
1         case_intake_arm_1                 0                 0   
2         case_intake_arm_1                 0                 0   
          case_intake_arm_1                 1                 1   
3         case_intake_arm_1                 0                 0   
4         case_intake_arm_1                 0                 0   
          case_intake_arm_1                 1                 1   
5         case_intake_arm_1                 0                 0   
6         case_intake_arm_1                 0                 0   

                             symptoms_exp___6  symptoms_exp___7  \
record_id redcap_event_name                                       
1         case_intake_arm_1                 0                 1   
2         case_intake_arm_1                 0                 0   
          case_intake_arm_1                 0                 1   
3         case_intake_arm_1                 0                 1   
4         case_intake_arm_1                 0                 0   
          case_intake_arm_1                 0                 1   
5         case_intake_arm_1                 0                 0   
6         case_intake_arm_1                 1                 1   

                             symptoms_exp___8  symptoms_exp___9  \
record_id redcap_event_name                                       
1         case_intake_arm_1                 0                 0   
2         case_intake_arm_1                 1                 0   
          case_intake_arm_1                 1                 1   
3         case_intake_arm_1                 0                 0   
4         ca

**Note:** if `record_id` is not specified, all fields that make up the unique key will still be exported.

:::

## Export Data as CSV (PyCap Only) {#sec-csv}

::: {.panel-tabset}

#### Python

In @sec-labeled and @sec-label_reports, it was noted that to export labeled headers, the data needs to be exported as a csv. See the example below on how to do this. 

In [91]:
from io import StringIO

data_csv = StringIO(project.export_records(records='2',
                       raw_or_label='label',
                       raw_or_label_headers='label', 
                       format_type='csv'))
df_csv = pd.read_csv(data_csv, sep=',')
df_csv

,Record ID,Event Name,Repeat Instrument,Repeat Instance,First Name,Last Name,Phone Number,ZIP Code,Date of birth,Age (years),...,Phone number of close contact,Email of close contact,Complete?.3,Supervisor Name,Supervisor email,Do you ever work in-person?,Date you last worked in-person?,Was this person at work while contagious? Symptom Onset Date: [case_intake_arm_1][symptom_onset],Was this person at work while contagious?,Complete?.4
0,2,Personal Info,NaN,NaN,Jane,Doe,(999) 999-9999,98105.0,1994-06-29,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Boss,fake_email@gmail.com,Yes,2023-10-10,Yes,NaN,Complete
2,2,Case Intake,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Case Intake,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Notifications,Close Contacts,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,Notifications,Close Contacts,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,NaN,Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN


:::

## Creating a Reference Class (REDCapR only)

::: {.panel-tabset}

#### R

This [Reference Class](https://cran.r-project.org/web/packages/REDCapR/REDCapR.pdf) represents a REDCap project. Once some values are set that are specific to a REDCap project (such as the URI and token), later calls are less verbose (such as reading and writing data).

**First, define the project:**

In [92]:
%%capture
%%R
project <- REDCapR::redcap_project$new(redcap_uri=url, token=token)
ds_all <- project$read()

In [93]:
%%capture
%%R
data <- project$read(fields = c("record_id", "gender", "first_name"))$data

::: {.content-hidden when-format="html"}

In [48]:
%%capture
%%R

data_tbl <- gt(data)
gt::gtsave(data_tbl, filename = 'export_records7.html', path = "./files/export_files/")

:::

**Pull the `record_id` for all 'female' records:**

In [58]:
%%R

record_of_females <- data$record_id[data$gender=='0']
record_of_females

 [1] NA NA  2 NA NA NA NA NA  4 NA NA NA NA NA  6 NA NA


:::

## Clean Checkbox Choices (REDCapR Only)

::: {.panel-tabset}

#### R

REDCapR has a `checkbox_choices` function that can be used to neatly list all answer options for a checkbox field. 

First pull the metadata:

In [94]:
%%capture
%%R

metadata <- redcap_metadata_read(
    redcap_uri = url, 
    token = token
)$data

Select the checkbox field that you would like to view:

In [97]:
%%R
symptoms_exp <- metadata[metadata$field_name == "symptoms_exp",]$select_choices_or_calculations
symptoms_exp_list <- REDCapR::checkbox_choices(select_choices=symptoms_exp)
symptoms_exp_list

# A tibble: 11 x 2
   id    label              
   <chr> <chr>              
 1 1     Sore Throat        
 2 2     Cough              
 3 3     Shortness of Breath
 4 4     Chest Pain         
 5 5     Headache           
 6 6     Runny Nose         
 7 7     Congestion         
 8 8     Fever              
 9 9     Body Aches         
10 10    Nausea/Vomiting    
11 11    Diarrhea           


:::

## REDCap Constants (REDCapR Only)

::: {.panel-tabset}

#### R

You can quickly search for the numerical values of several 'constants' within REDCap. For example, when exporting data from REDCap, all instruments end with a variable called `<form_name>_complete` that when exported as raw data will take the values of 0, 1 or 2. You can see what each numerical value means by using `constant("form_incomplete")` and `constant("form_complete")`.  

Other constants across all REDCap projects include the values for the various user rights settings. For a full list of constants available, refer to the following [documentation.](https://ouhscbbmc.github.io/REDCapR/reference/constant.html)

**View the `<form_name>_complete` REDCap constant values:**

In [81]:
%%R
REDCapR::constant(c(
  "form_incomplete",
  "form_complete",
  "form_unverified"
))

[1] 0 2 1


:::